# 🚦 Indian Traffic Sign Detection - YOLOv8 Training

**Author:** Himanshu Sharma  
**Project:** Smart Traffic Detection System  
**Model:** YOLOv8 Nano (Optimized for Browser Deployment)

---

## 📋 Prerequisites

Before running this notebook:

1. **Download Dataset from Roboflow:**
   - Go to: https://universe.roboflow.com/datacluster-labs/indian-traffic-sign
   - Click "Download" → Select "YOLOv8" format
   - Download as ZIP

2. **Upload to Google Drive:**
   - Upload the ZIP file to `My Drive/datasets/`
   - Name it: `indian-traffic-signs.zip`

3. **Enable GPU:**
   - Go to Runtime → Change runtime type → T4 GPU

---

## Cell 1: Mount Google Drive & Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Ultralytics (YOLOv8)
!pip install ultralytics -q

# Verify GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Cell 2: Extract Dataset

In [ ]:
import os
import zipfile

# Path to your uploaded dataset
DATASET_ZIP = '/content/drive/MyDrive/datasets/indian-traffic-signs.zip'
DATASET_DIR = '/content/dataset'

# Check if ZIP exists
if not os.path.exists(DATASET_ZIP):
    print("❌ ERROR: Dataset ZIP not found!")
    print(f"   Expected path: {DATASET_ZIP}")
    print("")
    print("📝 Instructions:")
    print("   1. Download dataset from Roboflow (YOLOv8 format)")
    print("   2. Upload ZIP to Google Drive: My Drive/datasets/indian-traffic-signs.zip")
    print("   3. Re-run this cell")
else:
    # Extract dataset
    print("📦 Extracting dataset...")
    with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
    print(f"✅ Dataset extracted to: {DATASET_DIR}")

    # Show dataset structure
    print("\n📁 Dataset structure:")
    for root, dirs, files in os.walk(DATASET_DIR):
        level = root.replace(DATASET_DIR, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        if level < 2:  # Only show first 2 levels
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:  # Show max 5 files
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... and {len(files)-5} more files")

## Cell 3: Find data.yaml Path

In [ ]:
import glob

# Find data.yaml file
yaml_files = glob.glob(f"{DATASET_DIR}/**/data.yaml", recursive=True)

if not yaml_files:
    print("❌ data.yaml not found! Check dataset format.")
else:
    DATA_YAML = yaml_files[0]
    print(f"✅ Found data.yaml: {DATA_YAML}")

    # Show content
    print("\n📝 Dataset configuration:")
    with open(DATA_YAML, 'r') as f:
        content = f.read()
        print(content)

    # Count classes
    import yaml
    with open(DATA_YAML, 'r') as f:
        data = yaml.safe_load(f)
        if 'names' in data:
            print(f"\n🏷️ Number of classes: {len(data['names'])}")
            print(f"📋 Classes: {list(data['names'].values()) if isinstance(data['names'], dict) else data['names']}")

## Cell 4: Train YOLOv8 Nano Model

**Training Parameters:**
- **Model:** yolov8n.pt (Nano ~3MB - optimized for browser)
- **Epochs:** 50 (balanced for small dataset)
- **Image Size:** 512 (faster than 640, minimal accuracy loss)
- **Patience:** 10 (early stopping)
- **Estimated Time:** 45-60 minutes

In [ ]:
from ultralytics import YOLO
import os

# Load YOLOv8 Nano (smallest, fastest - perfect for browser)
model = YOLO('yolov8n.pt')

print("🚀 Starting training...")
print("⏱️ Estimated time: 45-60 minutes")
print("")

# Train the model
results = model.train(
    data=DATA_YAML,
    epochs=50,           # Good for small dataset
    imgsz=512,           # Faster training
    batch=16,            # Fits T4 GPU memory
    patience=10,         # Early stopping
    device=0,            # Use GPU
    workers=2,           # Optimal for Colab
    project='/content/runs',
    name='indian_traffic_signs',
    exist_ok=True,
    # Augmentation (helps with small dataset)
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    flipud=0.0,          # Don't flip signs upside down
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
)

print("\n✅ Training complete!")

## Cell 5: Evaluate Model Performance

In [ ]:
# Load best model
best_model_path = '/content/runs/indian_traffic_signs/weights/best.pt'
model = YOLO(best_model_path)

# Validate on test set
print("📊 Evaluating model...")
metrics = model.val()

print("\n📈 Results:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

## Cell 6: Export to TensorFlow.js (For Browser)

This converts the PyTorch model to TensorFlow.js format for browser deployment.

In [ ]:
# Export to TensorFlow.js format
print("📦 Exporting to TensorFlow.js...")
print("⏱️ This may take 5-10 minutes...")

model.export(
    format='tfjs',
    imgsz=512,
    simplify=True,
)

print("\n✅ Export complete!")

# Find exported files
import os
export_dir = '/content/runs/indian_traffic_signs/weights/best_web_model'

if os.path.exists(export_dir):
    print(f"\n📁 Exported files in: {export_dir}")
    for f in os.listdir(export_dir):
        size = os.path.getsize(os.path.join(export_dir, f))
        print(f"   {f}: {size/1024:.1f} KB")
else:
    # Try alternative path
    alt_paths = glob.glob('/content/runs/**/*web_model*', recursive=True)
    if alt_paths:
        export_dir = alt_paths[0]
        print(f"\n📁 Found export at: {export_dir}")

## Cell 7: Create Class Names JSON

In [ ]:
import json
import yaml

# Read class names from data.yaml
with open(DATA_YAML, 'r') as f:
    data = yaml.safe_load(f)

# Get class names
if 'names' in data:
    if isinstance(data['names'], dict):
        class_names = list(data['names'].values())
    else:
        class_names = data['names']

# Save as JSON
class_json = {
    'class_names': class_names,
    'model_info': {
        'name': 'Indian Traffic Sign Detector',
        'version': '1.0.0',
        'base_model': 'yolov8n',
        'input_size': 512,
        'num_classes': len(class_names)
    }
}

# Save in export directory
class_file = f'{export_dir}/class_names.json'
with open(class_file, 'w') as f:
    json.dump(class_json, f, indent=2)

print(f"✅ Saved class names to: {class_file}")
print(f"📋 {len(class_names)} classes: {class_names[:10]}...")

## Cell 8: Copy to Google Drive & Download

In [ ]:
import shutil
import os

# Create output directory in Google Drive
DRIVE_OUTPUT = '/content/drive/MyDrive/models/indian_traffic_signs_tfjs'
os.makedirs(DRIVE_OUTPUT, exist_ok=True)

# Copy TF.js model files
print("📦 Copying model to Google Drive...")

for f in os.listdir(export_dir):
    src = os.path.join(export_dir, f)
    dst = os.path.join(DRIVE_OUTPUT, f)
    shutil.copy2(src, dst)
    print(f"   ✅ {f}")

# Also copy best.pt (PyTorch weights)
shutil.copy2(best_model_path, f'{DRIVE_OUTPUT}/best.pt')
print(f"   ✅ best.pt")

print(f"\n✅ All files copied to: {DRIVE_OUTPUT}")
print("\n📥 Download instructions:")
print("   1. Go to Google Drive")
print("   2. Navigate to: My Drive/models/indian_traffic_signs_tfjs/")
print("   3. Download all files")
print("   4. Place in: frontend/public/models/traffic_signs/")

## Cell 9: Create ZIP for Easy Download

In [ ]:
import shutil
from google.colab import files

# Create ZIP file
ZIP_NAME = 'indian_traffic_signs_model'
shutil.make_archive(f'/content/{ZIP_NAME}', 'zip', export_dir)

# Copy to Drive
shutil.copy2(f'/content/{ZIP_NAME}.zip', f'{DRIVE_OUTPUT}/{ZIP_NAME}.zip')

print(f"✅ Created: {ZIP_NAME}.zip")
print(f"📍 Location: {DRIVE_OUTPUT}/{ZIP_NAME}.zip")
print("\n⬇️ Click below to download directly:")

# Offer direct download
files.download(f'/content/{ZIP_NAME}.zip')

## ✅ Done! Next Steps

1. **Download the ZIP file** (should auto-download from previous cell)

2. **Extract to your project:**
   ```
   frontend/public/models/traffic_signs/
   ├── model.json
   ├── group1-shard1of*.bin
   └── class_names.json
   ```

3. **Tell your AI assistant** that training is complete

---

### 📊 Model Stats
- **Size:** ~3-5 MB (browser-friendly)
- **Input:** 512x512 images
- **Speed:** ~50ms per image on modern browser
- **Classes:** 85 Indian traffic signs